In [16]:
# Import libraries and dependencies

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pandas as pd

In [17]:
# Create an engine to a sqlite database

engine = create_engine("sqlite:///hawaii.sqlite")

In [18]:
# Create a connection to the engine

conn = engine.connect()

In [19]:
# Have the tables show up by being mirrored into the Jupyter Notebook from the database
Base = declarative_base()

In [20]:
class Measurements(Base):
    __tablename__ = 'measurements'
    station = Column(String(200), primary_key=True)
    date = Column(String(200))
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    

class Station(Base):
    __tablename__ = 'station'
    station = Column(String(200),  primary_key=True)
    name = Column(String(200))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [21]:
# Create engine 

Base.metadata.create_all(engine)

In [22]:
# Create Dataframes

measurements_hawaii = pd.read_csv('cleaned_hawaii_measurements.csv')
stations_hawaii = pd.read_csv('hawaii_stations.csv')

In [23]:
# Write data using "orient" to convert DF into dictionaries

weather_df = measurements_hawaii.to_dict(orient='records')
station_df = stations_hawaii.to_dict(orient='records')
weather_df[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [24]:
# Reflect the tables from SQLalchemy

metadata = MetaData(bind=engine)
metadata.reflect()

In [25]:
# Create tables

weather_hawaii = sqlalchemy.Table('measurements', metadata, autoload=True)
station_hawaii = sqlalchemy.Table('station', metadata, autoload=True)

In [26]:
# Delete duplicate data in tables

conn.execute(weather_hawaii.delete())
conn.execute(station_hawaii.delete())

In [27]:
# Insert data into tables

conn.execute(weather_hawaii.insert(), weather_df)
conn.execute(station_hawaii.insert(), station_df)

In [28]:
# Test that the insert works by fetching the first 5 rows. 

conn.execute("SELECT * FROM measurements LIMIT 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [29]:
conn.execute("SELECT * FROM station").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]